# Kerchunk JSON File Recipe
The standard format for Kerchunk files is JSON for storing references to archive data chunks. This requires fsspec and xarray - although these do not need to be the latest, there is no need to not use fsspec 2023.6.0+ and xarray 2023.8.0+ as these will also work with Parquet.

In [ ]:
import fsspec
import xarray as xr

Open a virtual filesystem object from fsspec, providing the kerchunk file as a 'reference' type file. Note that if the file is compressed using zstd as some kerchunk files are, the compression needs to be set to 'zstd'/'zst'

In [ ]:
kfile = '../../example-kerchunk/kerchunk-kr1.0.json'
mapper = fsspec.get_mapper('reference://',fo=kfile, backend_kwargs={'compression':None)

Then we can open a virtual xarray dataset object to plot or perform some processing.

In [ ]:
ds = xr.open_zarr(mapper, consolidated=False)

Any plotting can then be done with this object.

In [ ]:
ds['example-variable'].mean(dim='time').plot()